In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
movie = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [ ]:
movie.head()

# original title

# genre
# keywords
# overview
# cast(but only top 3)
# crew (only director)

In [ ]:
movie.shape

In [ ]:
credits.head()

In [ ]:
credits.iloc[0]['crew']

In [ ]:
final = pd.merge(movie,credits,left_on='id',right_on='movie_id')[['id','original_title','genres','keywords','overview','cast','crew']]

In [ ]:
final.head()

In [ ]:
final.iloc[0]['genres']

In [ ]:
L1 = []
for i in ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'):
    L1.append(i['name'])
print(L1)

In [ ]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    L1 = L[:]
    L.clear()
    return L1
        

In [ ]:
final['genres'] = final['genres'].apply(convert)

In [ ]:
final.head()

In [ ]:
final['keywords'] = final['keywords'].apply(convert)

In [ ]:
final['overview'] = final['overview'].apply(lambda x: x.split())

In [ ]:
final.isnull().sum()

In [ ]:
final.dropna(subset=['overview'],inplace=True)

In [ ]:
def fetch_cast(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        else:
            break
        counter+=1
    L1 = L[:]
    L.clear()
    return L1
        
    
        

In [ ]:
final['cast'] = final['cast'].apply(fetch_cast)

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    L1 = L[:]
    L.clear()
    return L1
            

In [ ]:
final['crew'] = final['crew'].apply(fetch_director)

In [ ]:
final['tags'] = final['genres'] + final['keywords'] + final['overview'] + final['cast'] + final['crew']

In [ ]:
final.head()

In [ ]:
def remove_space(L):
    L1 = []
    for i in L:
        L1.append(i.replace(' ',''))
        
    L2 = L1[:]
    L1.clear()
        
    return L2

In [ ]:
final['tags'] = final['tags'].apply(remove_space)

In [ ]:
final.head()

In [ ]:
final['tags'] = final['tags'].apply(lambda x:" ".join(x))

In [ ]:
final.head()

In [ ]:
final['tags'][3]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
final['tags'] = final['tags'].apply(lambda x:x.lower())

In [ ]:
cv = CountVectorizer(stop_words = 'english',max_features=10000)

In [ ]:
X = cv.fit_transform(final['tags']).toarray()

In [ ]:
X.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_matrix = cosine_similarity(X)

In [ ]:
similarity_matrix[1]

In [ ]:
def recommend(movie_name):
    
    movie_index = final[final['original_title'] == movie_name].index[0]
    L = sorted(list(enumerate(similarity_matrix[movie_index])),reverse=True, key=lambda x:x[1])
    
    for i in L[1:6]:
        print(final.iloc[i[0]]['original_title'])
        
    

In [ ]:
recommend('King Kong')

In [ ]:
final.iloc[15:25]

In [ ]:
final.iloc[506]['original_title']

In [ ]:
sorted(list(enumerate(similarity_matrix[0])), key=lambda x:x[1])[1:6]